# Data - How Institutions of Higher Education Can Mine The Key Ingredient of  A Successful Analytics Program

The development of data analytics programs that foster robust decision support systems have become a key component of strategic initiatives for institutions of higher-education. However, college and university leadership teams remain unsure of how to effectively incorporate analytics into the institution's operations.

The object of this project is to explore the power of a predictive system based on the notion that at the end of each semester we can diagnose the chances of a student graduating in 4 years based on all available information up to that point. 

One main objective is to identify an information saturation point upon which actionable intervention is implemented. The hypothesis is that there exists a local maxima saturation point which provides the most appropriate intervention point.

# Tools

In [99]:
# General
from __future__ import print_function, division
import sys

# Database
import cx_Oracle
from sqlalchemy import create_engine
from getpass import getpass

# Tools
import pandas as pd
import seaborn as sns
import random
import string
from builtins import range
import matplotlib.pyplot as plt
import numpy as np
import time
#import pandas_profiling

pd.options.display.max_columns = None

# Analytics
#import pymc3 as pm
from scipy.stats import beta

#IMBALANCED DATA
# from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler

#SciKitLearn Models
# from sklearn.linear_model import LogisticRegression, ElasticNetCV, SGDClassifier
# from sklearn import svm
# from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier, AdaBoostClassifier,VotingClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
# from sklearn.naive_bayes import GaussianNB
# from sklearn.neural_network import MLPClassifier #(wait for scikit release 18.0)
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LassoCV
# from sklearn.feature_selection import SelectFromModel

#XgBoost Model ###################################################################################
# import os
# mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
# os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost as xgb
##################################################################################################

#MODEL SELECTION, #EVALUATION METRICS
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn import preprocessing

# Academic Data: Performance & Demographics

#### Grades Data
First we obtain the students grades along with a few variables. The goal is to aggregate the grade records to a semester summary.

In [2]:
username = input('Enter username: ')

password = getpass(prompt='Enter password: ')

Enter username: jbraswell
Enter password: ········


In [3]:
service_name = 'iraarchd'
host = 'ira-oradb01d.its.csulb.edu'
port = '1521'
grades_query = 'grd_msk.sql'
dem_query = 'dae_msk.sql'

def db_query(username, password, service_name, host, port, query):

    dsn = cx_Oracle.makedsn(host, port, service_name=service_name)

    cstr = 'oracle://{user}:{password}@{dsn}'.format(
        user=username,
        password=password,
        dsn=dsn
    )

    engine =  create_engine(
        cstr,
        convert_unicode=False,
        pool_recycle=10,
        pool_size=50,
    )

    with open(query, 'r') as f:
        data=f.read()#.replace('\n', '')
        
    return (data, engine)

In [4]:
grades, engine = db_query(username, password, service_name, host, port, grades_query)
grd = pd.read_sql(grades, engine)

demo, engine = db_query(username, password, service_name, host, port, dem_query)
dem = pd.read_sql(demo, engine)


print(grd.shape)
print(dem.shape)

C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\dialects\oracle\base.py:1316: SAWarning: Oracle compatibility version (18, 5, 0, 0, 0) is known to have a maximum identifier length of 128, rather than the historical default of 30. SQLAlchemy 1.4 will use 128 for this database; please set max_identifier_length=128 in create_engine() in order to test the application with this new length, or set to 30 in order to assure that 30 continues to be used.  In particular, pay close attention to the behavior of database migrations as dynamically generated names may change. See the section 'Max Identifier Lengths' in the SQLAlchemy Oracle dialect documentation for background.
  % ((self.server_version_info,))
C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\dialects\oracle\base.py:1273: SAWarning: Could not determine compatibility version: (cx_Oracle.DatabaseError) ORA-00942: table or view does not exist
[SQL: SELECT value FROM v$parameter WHERE name = 'compatible']
(Background on this e

(709796, 37)
(12251, 23)


## A first look at our data

In [5]:
grd.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709796 entries, 0 to 709795
Data columns (total 37 columns):
emplid                    709796 non-null object
cohort                    709796 non-null object
term_code                 709796 non-null object
class_term                709796 non-null object
session_code              709796 non-null object
eot_acad_career           709796 non-null object
eot_program_cd            709796 non-null object
eot_program               709796 non-null object
eot_college_cd            709742 non-null object
eot_college               709742 non-null object
eot_department_cd         709742 non-null object
eot_department            709742 non-null object
eot_acad_plan_cd          709796 non-null object
eot_acad_plan             709796 non-null object
class_college_code        709625 non-null object
class_college             709625 non-null object
class_dept_code           709625 non-null object
class_dept                709625 non-null object
class_p

In [6]:
dem.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12251 entries, 0 to 12250
Data columns (total 23 columns):
emplid                   12251 non-null object
dem_cohort               12251 non-null object
dem_diff_indx            8827 non-null float64
dae_emplid               12251 non-null int64
gender                   12251 non-null object
ethnicity                12251 non-null object
first_generation         12251 non-null object
dep_family_size          12251 non-null object
minority                 12251 non-null object
applicant_family_size    12251 non-null object
applicant_income         12251 non-null object
pell_tot_emplid          12251 non-null int64
pell_eligibility         6711 non-null object
esa_emplid               12251 non-null int64
act_comp                 4216 non-null float64
act_read                 4216 non-null float64
act_math                 4216 non-null float64
act_eng                  4212 non-null float64
act_sci                  4216 non-null float64
sa

In [7]:
grd.head()

,emplid,cohort,term_code,class_term,session_code,eot_acad_career,eot_program_cd,eot_program,eot_college_cd,eot_college,eot_department_cd,eot_department,eot_acad_plan_cd,eot_acad_plan,class_college_code,class_college,class_dept_code,class_dept,class_prefix,class_prefix_longdesc,class_catalog_nbr,class_section,class_descr,bcmp,bcmp_units_taken,online_course,units_taken,grading_basis,official_grade,earned_credit,units_attempted,grade_points,status_cd,ge_flag,division,gpa_flag,class_instruction_mode
0,7472,2124,2154,Fall 2015,1,Undergraduate,UGD,Undergraduate Degree,COTA,The Arts,ART,Art,ART_BA02U1,Art History BA,COLA,Liberal Arts,ENGL,English,ENGL,English,372,01,Comedy in the United States,0,0,Not_Online,3.0,GCR,C,Y,Y,6.0,E,GE,Upper-Division,Include_in_GPA,P
1,15442,2124,2154,Fall 2015,1,Undergraduate,UGD,Undergraduate Degree,CBA,Business Administration,MGMT,Management,MGMTBS03U1,Human Resources Management BS,CBA,Business Administration,I S,Information Systems,I S,Information Systems,310,11,Business Statistics I,0,0,Not_Online,3.0,GRD,C,Y,Y,6.0,E,Non_GE,Upper-Division,Include_in_GPA,P
2,1044,2124,2162,Spring 2016,1,Undergraduate,UGD,Undergraduate Degree,COTA,The Arts,MUS,Music,MUS_BM05U1,Performance BM,COTA,The Arts,MUS,Music,MUS,Music,272B,01,Jazz Theory II,0,0,Not_Online,2.0,GRD,A,Y,Y,6.0,E,Non_GE,Lower-Division,Include_in_GPA,P
3,15551,2124,2154,Fall 2015,1,Undergraduate,UGD,Undergraduate Degree,COLA,Liberal Arts,COMM,Communication Studies,COMMBA01U1,Communication Studies BA,CHHS,Health and Human Services,FCS,Family and Consumer Sciences,HFHM,Hospitality Management,170,01,Intro to Hospitality Mgt,0,0,Not_Online,3.0,GCR,A,Y,Y,9.0,E,Non_GE,Lower-Division,Include_in_GPA,P
4,8004,2124,2163,Summer 2016,SSI,Undergraduate,UGD,Undergraduate Degree,COLA,Liberal Arts,PSY,Psychology,PSY_BA01U1,Psychology BA,COLA,Liberal Arts,PSY,Psychology,PSY,Psychology,457,01,Psychology of Sexuality,0,0,Not_Online,3.0,GRD,C,Y,Y,6.0,E,Non_GE,Upper-Division,Include_in_GPA,P


In [8]:
dem.head()

,emplid,dem_cohort,dem_diff_indx,dae_emplid,gender,ethnicity,first_generation,dep_family_size,minority,applicant_family_size,applicant_income,pell_tot_emplid,pell_eligibility,esa_emplid,act_comp,act_read,act_math,act_eng,act_sci,sat_read,sat_math,sat_comp,gpa_hs
0,2,2124,NaN,2,F,CAUCASIAN,First Generation Student,NA,VISA NON U.S.,NA,NO RESPONSE,2,TRADITIONAL,2,NaN,NaN,NaN,NaN,NaN,530.0,610.0,1140.0,310.0
1,3,2104,50.0,3,F,CAUCASIAN,First Generation Student,7,MINORITY,NA,NO RESPONSE,3,TRADITIONAL,3,NaN,NaN,NaN,NaN,NaN,470.0,400.0,870.0,313.0
2,4,2124,38.0,4,M,LATINO/LATINA,First Generation Student,2,MINORITY,NA,NO RESPONSE,4,None,4,20.0,22.0,18.0,20.0,18.0,600.0,530.0,1130.0,305.0
3,5,2104,38.0,5,F,LATINO/LATINA,First Generation Student,4,MINORITY,NA,NO RESPONSE,5,None,5,NaN,NaN,NaN,NaN,NaN,470.0,590.0,1060.0,253.0
4,6,2124,48.0,6,M,VISA NON U.S.,First Generation Student,6,MINORITY,NA,NO RESPONSE,6,None,6,NaN,NaN,NaN,NaN,NaN,500.0,540.0,1040.0,341.0


In [9]:
dem[dem['emplid'] == '3']

,emplid,dem_cohort,dem_diff_indx,dae_emplid,gender,ethnicity,first_generation,dep_family_size,minority,applicant_family_size,applicant_income,pell_tot_emplid,pell_eligibility,esa_emplid,act_comp,act_read,act_math,act_eng,act_sci,sat_read,sat_math,sat_comp,gpa_hs
1,3,2104,50.0,3,F,CAUCASIAN,First Generation Student,7,MINORITY,NA,NO RESPONSE,3,TRADITIONAL,3,NaN,NaN,NaN,NaN,NaN,470.0,400.0,870.0,313.0


In [10]:
grd[grd['emplid'] == '3'].sort_values(by=['term_code'])

,emplid,cohort,term_code,class_term,session_code,eot_acad_career,eot_program_cd,eot_program,eot_college_cd,eot_college,eot_department_cd,eot_department,eot_acad_plan_cd,eot_acad_plan,class_college_code,class_college,class_dept_code,class_dept,class_prefix,class_prefix_longdesc,class_catalog_nbr,class_section,class_descr,bcmp,bcmp_units_taken,online_course,units_taken,grading_basis,official_grade,earned_credit,units_attempted,grade_points,status_cd,ge_flag,division,gpa_flag,class_instruction_mode
657605,3,2104,2104,Fall 2010,1,Undergraduate,UGD,Undergraduate Degree,CHHS,Health and Human Services,S W,Social Work,SW__PR00U1,Pre-Social Work,COLA,Liberal Arts,COMM,Communication Studies,COMM,Communication Studies,130,46,Essential Public Speaking,0,0,Not_Online,3.0,GCR,A,Y,Y,9.0,E,GE,Lower-Division,Include_in_GPA,P
648326,3,2104,2104,Fall 2010,1,Undergraduate,UGD,Undergraduate Degree,CHHS,Health and Human Services,S W,Social Work,SW__PR00U1,Pre-Social Work,UNPG,University Programs,S/I,Supplemental Instruction,S/I,Supplemental Instruction,60,14,Supplemental Instruction,0,0,Not_Online,1.0,NOG,D,Y,Y,0.0,E,Non_GE,Pre-Collegiate,Exclude_from_GPA,P
643598,3,2104,2104,Fall 2010,1,Undergraduate,UGD,Undergraduate Degree,CHHS,Health and Human Services,S W,Social Work,SW__PR00U1,Pre-Social Work,COLA,Liberal Arts,CWL,Comparative Wrld Literature,CLSC,Classics,101,02,Greek Mythology,0,0,Not_Online,3.0,GCR,C,Y,Y,6.0,E,GE,Lower-Division,Include_in_GPA,P
610162,3,2104,2104,Fall 2010,1,Undergraduate,UGD,Undergraduate Degree,CHHS,Health and Human Services,S W,Social Work,SW__PR00U1,Pre-Social Work,CNSM,Natural Sciences & Mathematics,MATH,Mathematics,MAPB,Mathematics Prebaccalaureate,1,04,Elem Algebra-Geometry,1,4,Not_Online,4.0,NOG,D,Y,Y,0.0,E,Non_GE,Pre-Collegiate,Exclude_from_GPA,P
595973,3,2104,2104,Fall 2010,1,Undergraduate,UGD,Undergraduate Degree,CHHS,Health and Human Services,S W,Social Work,SW__PR00U1,Pre-Social Work,UNPG,University Programs,EOP,Educational Opportunity Prog,EOP,Educational Opportunity Prog,100,17,EOP Orientation,0,0,Not_Online,2.0,GCR,C,Y,Y,4.0,E,Non_GE,Lower-Division,Include_in_GPA,P
668685,3,2104,2104,Fall 2010,1,Undergraduate,UGD,Undergraduate Degree,CHHS,Health and Human Services,S W,Social Work,SW__PR00U1,Pre-Social Work,COLA,Liberal Arts,CHLS,Chicano and Latino Studies,CHLS,Chicano & Latino Studies,1,06,Writing Skills,0,0,Not_Online,3.0,NOG,D,Y,Y,0.0,E,Non_GE,Pre-Collegiate,Exclude_from_GPA,P
669431,3,2104,2112,Spring 2011,1,Undergraduate,UGD,Undergraduate Degree,CHHS,Health and Human Services,S W,Social Work,SW__PR00U1,Pre-Social Work,CNSM,Natural Sciences & Mathematics,MATH,Mathematics,MAPB,Mathematics Prebaccalaureate,7,02,Basic Intermediate Algebra,1,3,Not_Online,3.0,NOG,NC,N,Y,0.0,E,Non_GE,Pre-Collegiate,Exclude_from_GPA,P
661765,3,2104,2112,Spring 2011,1,Undergraduate,UGD,Undergraduate Degree,CHHS,Health and Human Services,S W,Social Work,SW__PR00U1,Pre-Social Work,COLA,Liberal Arts,SOC,Sociology,SOC,Sociology,100,10,Principles of Sociology,0,0,Not_Online,3.0,GCR,C,Y,Y,6.0,E,GE,Lower-Division,Include_in_GPA,P
661103,3,2104,2112,Spring 2011,1,Undergraduate,UGD,Undergraduate Degree,CHHS,Health and Human Services,S W,Social Work,SW__PR00U1,Pre-Social Work,COLA,Liberal Arts,AFRS,Africana Studies,AFRS,Africana Studies,100,03,Composition,0,0,Not_Online,3.0,GCR,A,Y,Y,9.0,E,GE,Lower-Division,Include_in_GPA,P
659852,3,2104,2112,Spring 2011,1,Undergraduate,UGD,Undergraduate Degree,CHHS,Health and Human Services,S W,Social Work,SW__PR00U1,Pre-Social Work,COLA,Liberal Arts,PHIL,Philosophy,PHIL,Philosophy,160,09,Intro Ethics,0,0,Not_Online,3.0,GCR,W,N,Y,0.0,E,GE,Lower-Division,Include_in_GPA,P


In [11]:
#Change 'UNKNOWN' to more unique string to avoid having columns with same name after one-hot-encode

dem['first_generation'] = dem['first_generation'].apply(lambda x: 'First Generation Unknown' if x == 'UNKNOWN' else x)
dem['ethnicity'] = dem['ethnicity'].apply(lambda x: 'ETHNICITY UNKNOWN' if x == 'UNKNOWN' else x)

# Data Wrangling

##### Before doing anything it is good practice to split our datasets into Training and Testing sets.  We do this before any of our data wrangling to avoid any possible data leakage between the two datasets.

Ultimately, our observations are all based on students - so it is students that we need perform the split on.  We will use a 90/10 between train dataset and test dataset.

Our first step is to create a dataframe of just our student EMPLIDs.

Note to Team:  I think this should be done later when we actually do the train/test split

In [12]:
students = pd.DataFrame(grd['emplid'].unique(), columns=['emplid'])

print('there are {} students'.format(students.shape[0]))

there are 15497 students


##### Create the Grades Trainning and Validation Student Set

In [13]:
students_train, students_dev = train_test_split(students, test_size=0.10, random_state=42)

students_train = pd.DataFrame(students_train)

students_dev = pd.DataFrame(students_dev)

In [14]:
students_train.columns = ['EMPLID']
students_dev.columns = ['EMPLID']

In [15]:
students_train.head()

,EMPLID
9402,14105
8388,10741
5180,3045
5118,1713
1247,11130


## Preprocessing: One-Hot-Encode Letter Grades

In [16]:
#Get all our column names in upper case for ease of reference
grd.columns = map(str.upper, grd.columns)

Get Dummies Documentation

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

In [17]:
grd = pd.concat([grd,pd.get_dummies(grd['OFFICIAL_GRADE'], drop_first=True)], axis=1)

grd.shape

(709796, 50)

##### Create Variables to Calculate GPA

In [21]:
grd['OFFICIAL_GRADE'].apply(lambda x: None if x in ['AU','CR','NC','RD','RP','W','WE'] else 1).head(10)

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    NaN
8    1.0
9    1.0
Name: OFFICIAL_GRADE, dtype: float64

In [24]:
# These statements keep only those grade points, and units taken that counted towards GPA.  
#Notice what the lambda function above returned.

grd['GRADE_POINTS_IN_GPA'] = grd['GRADE_POINTS'] * grd['OFFICIAL_GRADE'].apply(
    lambda x: None if x in ['AU','CR','NC','RD','RP','W','WE'] else 1
)

grd['UNITS_IN_GPA'] = grd['UNITS_TAKEN'] * grd['OFFICIAL_GRADE'].apply(
    lambda x: None if x in ['AU','CR','NC','RD','RP','W','WE'] else 1
)

grd['UNITS_FOR_CREDIT'] = grd['UNITS_TAKEN'] * grd['OFFICIAL_GRADE'].apply(
    lambda x: None if x in ['AU','NC','RD','RP','W','WE'] else 1
)

#######################################################################################################################

#BCMP was a measure created in pre-processing that takes the grade points earned in 
#Biology, Chemistry,Math and Physics classes.  We expect these courses to have a strong relationship to timely graduation.

grd['BCMP_GRADE_POINTS_IN_GPA'] = grd['BCMP'] * grd['GRADE_POINTS'] * grd['OFFICIAL_GRADE'].apply(
    lambda x: None if x in ['AU','CR','NC','RD','RP','W','WE'] else 1
)

grd['BCMP_UNITS_IN_GPA'] = grd['BCMP_UNITS_TAKEN'] * grd['OFFICIAL_GRADE'].apply(
    lambda x: None if x in ['AU','CR','NC','RD','RP','W','WE'] else 1
)

grd['BCMP_UNITS_FOR_CREDIT'] = grd['BCMP'] * grd['UNITS_TAKEN'] * grd['OFFICIAL_GRADE'].apply(
    lambda x: None if x in ['AU','NC','RD','RP','W','WE'] else 1
)

In [27]:
#Creating a feature that measures how many credits were taken during Winter and Summer terms.
grd['SUMMER'] = (grd['TERM_CODE'].apply(lambda x: str(x)[-1]) == '3')* 1 * grd['UNITS_FOR_CREDIT']

grd['WINTER'] = (grd['TERM_CODE'].apply(lambda x: str(x)[-1]) == '1')* 1 * grd['UNITS_FOR_CREDIT']

##### Reduce the dataframe to variables of current interest.  This is the step in which domain knowledge become particularly vital.

In [28]:
grd = grd.sort_values(by=['EMPLID','TERM_CODE']).copy()[['COHORT', 
                 'EMPLID', 
                 'TERM_CODE',
                 'EOT_ACAD_PLAN_CD',
                 'GRADE_POINTS_IN_GPA',
                 'UNITS_TAKEN',
                 'UNITS_IN_GPA',
                 'UNITS_FOR_CREDIT',
                 'BCMP',
                 'BCMP_GRADE_POINTS_IN_GPA',
                 'BCMP_UNITS_TAKEN',
                 'BCMP_UNITS_IN_GPA',
                 'BCMP_UNITS_FOR_CREDIT',
                 'A', 
                 'AU', 
                 'B',
                 'C', 
                 'CR', 
                 'D', 
                 'F', 
                 'I', 
                 'NC', 
                 'RP', 
                 'W', 
                 'WE', 
                 'WU',
                 'SUMMER',
                 'WINTER']]

#### Aggregate and Reduce from Course Dimension to Term Dimension:  We want our data to be in the format of a single row for each term, rather than having a single row for each course in a term.  

In [29]:
aggregations = { 'GRADE_POINTS_IN_GPA':'sum',
                 'UNITS_TAKEN':'sum',
                 'UNITS_IN_GPA':'sum',
                 'UNITS_FOR_CREDIT':'sum',
                 'BCMP':'sum',
                 'BCMP_GRADE_POINTS_IN_GPA':'sum',
                 'BCMP_UNITS_TAKEN':'sum',
                 'BCMP_UNITS_IN_GPA':'sum',
                 'BCMP_UNITS_FOR_CREDIT':'sum',
                 'A':'sum', 
                 'AU':'sum', 
                 'B':'sum',
                 'C':'sum', 
                 'CR':'sum', 
                 'D':'sum', 
                 'F':'sum', 
                 'I':'sum', 
                 'NC':'sum', 
                 'RP':'sum', 
                 'W':'sum', 
                 'WE':'sum', 
                 'WU':'sum',
               'SUMMER':'sum',
               'WINTER':'sum'}

grouped_agg = grd.groupby(['COHORT','EMPLID','TERM_CODE','EOT_ACAD_PLAN_CD']).agg(aggregations).reset_index()

In [33]:
grouped_agg[grouped_agg['EMPLID'] == '3'].sort_values(by=['TERM_CODE'])

,COHORT,EMPLID,TERM_CODE,EOT_ACAD_PLAN_CD,GRADE_POINTS_IN_GPA,UNITS_TAKEN,UNITS_IN_GPA,UNITS_FOR_CREDIT,BCMP,BCMP_GRADE_POINTS_IN_GPA,BCMP_UNITS_TAKEN,BCMP_UNITS_IN_GPA,BCMP_UNITS_FOR_CREDIT,A,AU,B,C,CR,D,F,I,NC,RP,W,WE,WU,SUMMER,WINTER
47716,2104,3,2104,SW__PR00U1,19.0,16.0,16.0,16.0,1,0.0,4,4.0,4.0,1,0,0,2,0,3,0,0,0,0,0,0,0,0.0,0.0
47717,2104,3,2112,SW__PR00U1,21.0,16.0,10.0,10.0,1,0.0,3,0.0,0.0,1,0,0,2,0,1,0,0,1,0,1,0,0,0.0,0.0
47718,2104,3,2124,SW__PR00U1,36.0,12.0,12.0,12.0,1,6.0,3,3.0,3.0,2,0,1,1,0,0,0,0,0,0,0,0,0,0.0,0.0
47719,2104,3,2132,SW__PR00U1,36.0,12.0,12.0,12.0,0,0.0,0,0.0,0.0,2,0,1,1,0,0,0,0,0,0,0,0,0,0.0,0.0
47720,2104,3,2134,SOC_BA01U1,40.0,13.0,13.0,13.0,2,13.0,4,4.0,4.0,2,0,2,1,0,0,0,0,0,0,0,0,0,0.0,0.0
47721,2104,3,2142,SOC_BA01U1,48.0,13.0,13.0,13.0,0,0.0,0,0.0,0.0,1,0,3,0,0,0,0,0,0,0,0,0,0,0.0,0.0
47722,2104,3,2144,SOC_BA01U1,51.0,15.0,15.0,15.0,0,0.0,0,0.0,0.0,3,0,2,0,0,0,0,0,0,0,0,0,0,0.0,0.0
47723,2104,3,2152,SOC_BA01U1,54.0,15.0,15.0,15.0,0,0.0,0,0.0,0.0,2,0,3,0,0,0,0,0,0,0,0,0,0,0.0,0.0
47724,2104,3,2154,SOC_BA01U1,52.0,13.0,13.0,13.0,0,0.0,0,0.0,0.0,0,0,5,0,0,0,0,0,0,0,0,0,0,0.0,0.0


#### We now want to create a cummulative sum of Grade Points and GPA Units.

In [34]:
grouped_cumsum = grd[['EMPLID',
                             'TERM_CODE',
                             'EOT_ACAD_PLAN_CD',
                             'COHORT',
                             'GRADE_POINTS_IN_GPA',
                             'UNITS_TAKEN',
                             'UNITS_IN_GPA',
                             'UNITS_FOR_CREDIT',
                             'BCMP',
                             'BCMP_GRADE_POINTS_IN_GPA',
                             'BCMP_UNITS_TAKEN',
                             'BCMP_UNITS_IN_GPA',
                             'BCMP_UNITS_FOR_CREDIT',
                             'SUMMER',
                             'WINTER']].groupby(['EMPLID',
                                                 'TERM_CODE',
                                                 'COHORT',
                                                 'EOT_ACAD_PLAN_CD',]).sum().groupby(level=[0]).cumsum().reset_index()

grouped_cumsum = grouped_cumsum.add_prefix('CUM_')

grd = pd.concat([grouped_agg,grouped_cumsum],axis=1)

grd.drop(['CUM_EMPLID','CUM_TERM_CODE','CUM_COHORT','CUM_EOT_ACAD_PLAN_CD'],axis=1,inplace=True)

#### Create Term and Cummulative GPA

Note to team.  I think this should be earlier.  Before the aggregations and cumulations.

In [36]:
grd['TERM_GPA'] = grd['GRADE_POINTS_IN_GPA'] / grd['UNITS_IN_GPA']

grd['CUM_GPA'] = grd['CUM_GRADE_POINTS_IN_GPA'] / grd['CUM_UNITS_IN_GPA']

grd['BCMP_TERM_GPA'] = grd['BCMP_GRADE_POINTS_IN_GPA'] / grd['BCMP_UNITS_IN_GPA']

grd['BCMP_CUM_GPA'] = grd['CUM_BCMP_GRADE_POINTS_IN_GPA'] / grd['CUM_BCMP_UNITS_IN_GPA']

##### Check for NaN values since division by 0 is possible:

In [37]:
grd.isnull().sum()

COHORT                              0
EMPLID                              0
TERM_CODE                           0
EOT_ACAD_PLAN_CD                    0
GRADE_POINTS_IN_GPA                 0
UNITS_TAKEN                         0
UNITS_IN_GPA                        0
UNITS_FOR_CREDIT                    0
BCMP                                0
BCMP_GRADE_POINTS_IN_GPA            0
BCMP_UNITS_TAKEN                    0
BCMP_UNITS_IN_GPA                   0
BCMP_UNITS_FOR_CREDIT               0
A                                   0
AU                                  0
B                                   0
C                                   0
CR                                  0
D                                   0
F                                   0
I                                   0
NC                                  0
RP                                  0
W                                   0
WE                                  0
WU                                  0
SUMMER      

##### Impute missing Term and Cum GPA

In this instance it is reasonable to set the Term GPA and CumGPA to zero since NaNs result from Units in GPA and Cum Units in GPA being zero. This means either the student had no Units in GPA for a given term or the Cum Units in GPA was zero since the student failed to pass units in the initial term

In [38]:
grd.fillna(0, inplace=True)

# The Load Index

#### Firt we need to find a convenient way to calculate the number of terms that have passed since the student arrived

Start with difference in terms.

$The \ \ pattern \ \  of \ \ the \ \ difference \ \  d \ \ 0,7,8,9,10,17,18,19,20,... \ \ $

$Pattern \ \ when \ \ d \ \ mod \ \ 10 \ \ is \ \ 0$    $$N = 2\dfrac{d}{10} + 1 = \dfrac{d}{5} + 1$$

Pattern when d mod 10 is 8    $$N = 2\left(\dfrac{d - 8}{10} + 1\right) = \dfrac{d + 2}{5}$$

In [40]:

#Create the term difference 'd'
grd['TERM_DIFF'] = pd.to_numeric(grd['TERM_CODE']) - pd.to_numeric(grd['COHORT'])


# SEMESTER INDICATOR #
grd['N'] = grd['TERM_DIFF'].apply(lambda x: int(x/5+1) if x%10 == 0 else int((x + 2)/5) )


#How many units would a person needed to have taken at term n in order to be on track for 4 year graduation.
grd['PRESCRIBED_UNITS'] = grd['N'] * 15

In [44]:
grd[grd['EMPLID'] == '3'][['EMPLID','TERM_CODE','N','PRESCRIBED_UNITS']].head(19)

,EMPLID,TERM_CODE,N,PRESCRIBED_UNITS
47716,3,2104,1,15
47717,3,2112,2,30
47718,3,2124,5,75
47719,3,2132,6,90
47720,3,2134,7,105
47721,3,2142,8,120
47722,3,2144,9,135
47723,3,2152,10,150
47724,3,2154,11,165


#### Calculating the Load Index

$L_s = \dfrac{\sum_{i=1}^{k_s} u_i}{U_s}$

$u_i: units\ \ earned \ \ by \ \ taking \ \ class \ \ i.$

$k_s: number \ \ of \ \ classes \ \ taken \ \ in \ \ semester \ \ s.$

$U_s: number \ \ of \ \ units \ \ prescribed \ \ to \ \ be \ \ earned \ \ by \ \ semester \ \ s.$

In [45]:

grd['LOAD_INDEX'] = grd['CUM_UNITS_FOR_CREDIT'] / grd['PRESCRIBED_UNITS']

grd['COMPLETION_RATE'] = grd['UNITS_FOR_CREDIT'] / grd['UNITS_TAKEN']

## Create The Exclusive Load Index 

We are counting up the Units taken for credit only after the student arrived on campus.

In [47]:
grd['UNITS_FOR_CREDIT_EXCLUDE'] = (grd['TERM_DIFF'] >= 0) * grd['UNITS_FOR_CREDIT']

grouped_cumsum = grd[['EMPLID',
                      'TERM_CODE',
                      'COHORT',
                      'UNITS_FOR_CREDIT_EXCLUDE']].groupby(['EMPLID',
                                                                 'TERM_CODE',
                                                                 'COHORT']).sum().groupby(level=[0]).cumsum().reset_index()

grouped_cumsum = grouped_cumsum.add_prefix('CUM_')

grd = pd.concat([grd,grouped_cumsum],axis=1)

grd['N_EXCLUDE'] = grd['TERM_DIFF'].apply(lambda x: 0 if x < 0 else(int(x/5+1) if int(repr(x)[-1]) == 0 else int((x + 2)/5) ))

grd['PRESCRIBED_UNITS_EXCLUDE'] = grd['N_EXCLUDE'] * 15

grd['LOAD_INDEX_EXCLUDE'] = grd['CUM_UNITS_FOR_CREDIT_EXCLUDE'] / grd['PRESCRIBED_UNITS_EXCLUDE']

In [48]:
grd.head()

,COHORT,EMPLID,TERM_CODE,EOT_ACAD_PLAN_CD,GRADE_POINTS_IN_GPA,UNITS_TAKEN,UNITS_IN_GPA,UNITS_FOR_CREDIT,BCMP,BCMP_GRADE_POINTS_IN_GPA,BCMP_UNITS_TAKEN,BCMP_UNITS_IN_GPA,BCMP_UNITS_FOR_CREDIT,A,AU,B,C,CR,D,F,I,NC,RP,W,WE,WU,SUMMER,WINTER,CUM_GRADE_POINTS_IN_GPA,CUM_UNITS_TAKEN,CUM_UNITS_IN_GPA,CUM_UNITS_FOR_CREDIT,CUM_BCMP,CUM_BCMP_GRADE_POINTS_IN_GPA,CUM_BCMP_UNITS_TAKEN,CUM_BCMP_UNITS_IN_GPA,CUM_BCMP_UNITS_FOR_CREDIT,CUM_SUMMER,CUM_WINTER,TERM_GPA,CUM_GPA,BCMP_TERM_GPA,BCMP_CUM_GPA,TERM_DIFF,N,PRESCRIBED_UNITS,LOAD_INDEX,COMPLETION_RATE,UNITS_FOR_CREDIT_EXCLUDE,CUM_EMPLID,CUM_TERM_CODE,CUM_COHORT,CUM_UNITS_FOR_CREDIT_EXCLUDE,N_EXCLUDE,PRESCRIBED_UNITS_EXCLUDE,LOAD_INDEX_EXCLUDE
0,2094,1,2104,NDUGOT00U1,10.0,13.0,7.0,10.0,0,0.0,0,0.0,0.0,0,0,1,1,1,1,0,0,0,0,1,0,0,0.0,0.0,10.0,13.0,7.0,10.0,0,0.0,0,0.0,0.0,0.0,0.0,1.428571,1.428571,0.0,0.000000,10,3,45,0.222222,0.769231,10.0,1,2104,2094,10.0,3,45,0.222222
1,2094,1,2112,NDUGOT00U1,14.0,13.0,7.0,13.0,3,8.0,7,4.0,7.0,0,0,0,2,2,0,0,0,0,0,0,0,0,0.0,0.0,24.0,26.0,14.0,23.0,3,8.0,7,4.0,7.0,0.0,0.0,2.000000,1.714286,2.0,2.000000,18,4,60,0.383333,1.000000,13.0,1,2112,2094,23.0,4,60,0.383333
2,2094,1,2114,NDUGOT00U1,34.0,13.0,13.0,13.0,2,18.0,6,6.0,6.0,0,0,2,3,0,0,0,0,0,0,0,0,0,0.0,0.0,58.0,39.0,27.0,36.0,5,26.0,13,10.0,13.0,0.0,0.0,2.615385,2.148148,3.0,2.600000,20,5,75,0.480000,1.000000,13.0,1,2114,2094,36.0,5,75,0.480000
3,2094,1,2122,NDUGOT00U1,20.0,9.0,9.0,9.0,2,8.0,4,4.0,4.0,2,0,0,2,0,0,0,0,0,0,0,0,0,0.0,0.0,78.0,48.0,36.0,45.0,7,34.0,17,14.0,17.0,0.0,0.0,2.222222,2.166667,2.0,2.428571,28,6,90,0.500000,1.000000,9.0,1,2122,2094,45.0,6,90,0.500000
4,2094,1,2124,NDUGOT00U1,18.0,12.0,8.0,12.0,4,8.0,8,4.0,8.0,0,0,1,2,1,0,0,0,0,0,0,0,0,0.0,0.0,96.0,60.0,44.0,57.0,11,42.0,25,18.0,25.0,0.0,0.0,2.250000,2.181818,2.0,2.333333,30,7,105,0.542857,1.000000,12.0,1,2124,2094,57.0,7,105,0.542857


## Create the Only Prior Load Index Indicator

In [49]:
grd['LOAD_INDEX_ONLY']=grd['LOAD_INDEX']-grd['LOAD_INDEX_EXCLUDE']

grouped_diff = grd[['EMPLID', 'LOAD_INDEX_ONLY']].groupby(['EMPLID']).transform(max).reset_index()

grd = grd.drop(columns='LOAD_INDEX_ONLY')

grd = pd.concat([grd,grouped_diff],axis=1)

In [50]:
grd[['LOAD_INDEX_ONLY','N']].head()

,LOAD_INDEX_ONLY,N
0,0.0,3
1,0.0,4
2,0.0,5
3,0.0,6
4,0.0,7


## Create DFW Variables



In [51]:
grd['DFW'] = grd['D'] + grd['F'] + grd['I'] + grd['NC'] + grd['W'] + grd['WE'] + grd['WU']

grd['DFW_RATE'] = grd['DFW']/grd['UNITS_TAKEN']

## Removing Variables

We have used these to calculate various metrics, and no longer need them for our analasys.  They can be dropped to clean up our model.

In [53]:
grd.drop(labels=['GRADE_POINTS_IN_GPA','UNITS_IN_GPA','BCMP_GRADE_POINTS_IN_GPA',
                'BCMP_UNITS_IN_GPA','CUM_GRADE_POINTS_IN_GPA', 
                'CUM_UNITS_IN_GPA', 'CUM_BCMP_GRADE_POINTS_IN_GPA', 'TERM_DIFF', 
                 'CUM_EMPLID', 'CUM_TERM_CODE', 'CUM_COHORT',
                 'CUM_UNITS_FOR_CREDIT_EXCLUDE', 'N_EXCLUDE', 'PRESCRIBED_UNITS_EXCLUDE', 'index'],axis=1, inplace=True)

## Exclude Rows Before $T_0$

In [55]:
grd = grd[grd['COHORT'] <= grd['TERM_CODE']]

##### The Completion Rate calculation generated a few NaNs

Setting these NaNs to zero is appropriate since they result from dividing by zero (no units taken)

In [56]:
grd.fillna(0, inplace=True)

## Demographic Data

In [57]:
dem.columns = map(str.upper, dem.columns)
dem.columns

Index(['EMPLID', 'DEM_COHORT', 'DEM_DIFF_INDX', 'DAE_EMPLID', 'GENDER',
       'ETHNICITY', 'FIRST_GENERATION', 'DEP_FAMILY_SIZE', 'MINORITY',
       'APPLICANT_FAMILY_SIZE', 'APPLICANT_INCOME', 'PELL_TOT_EMPLID',
       'PELL_ELIGIBILITY', 'ESA_EMPLID', 'ACT_COMP', 'ACT_READ', 'ACT_MATH',
       'ACT_ENG', 'ACT_SCI', 'SAT_READ', 'SAT_MATH', 'SAT_COMP', 'GPA_HS'],
      dtype='object')

### One-Hot-Encode Demographics

In [58]:
dem = pd.concat([dem,
                pd.get_dummies(dem['GENDER'], drop_first=True, prefix='GENDR'),
                pd.get_dummies(dem['ETHNICITY'], drop_first=False),
                pd.get_dummies(dem['FIRST_GENERATION'], drop_first=False),
                pd.get_dummies(dem['DEP_FAMILY_SIZE'], drop_first=False, prefix='DEP_FAM'),
                pd.get_dummies(dem['MINORITY'], drop_first=False, prefix='URM'), 
                pd.get_dummies(dem['APPLICANT_FAMILY_SIZE'], drop_first=False, prefix='APP_FAM'),
                pd.get_dummies(dem['APPLICANT_INCOME'], drop_first=False, prefix='INCM'),
                pd.get_dummies(dem['PELL_ELIGIBILITY'], drop_first=False, prefix='PELL')], axis=1)

In [59]:
dem.columns = map(str.upper, dem.columns)
dem.columns

Index(['EMPLID', 'DEM_COHORT', 'DEM_DIFF_INDX', 'DAE_EMPLID', 'GENDER',
       'ETHNICITY', 'FIRST_GENERATION', 'DEP_FAMILY_SIZE', 'MINORITY',
       'APPLICANT_FAMILY_SIZE', 'APPLICANT_INCOME', 'PELL_TOT_EMPLID',
       'PELL_ELIGIBILITY', 'ESA_EMPLID', 'ACT_COMP', 'ACT_READ', 'ACT_MATH',
       'ACT_ENG', 'ACT_SCI', 'SAT_READ', 'SAT_MATH', 'SAT_COMP', 'GPA_HS',
       'GENDR_M', 'AFRICAN AMERICAN', 'ASIAN AMERICAN', 'CAUCASIAN',
       'ETHNICITY UNKNOWN', 'LATINO/LATINA', 'NATIVE AMERICAN',
       'PACIFIC ISLANDER', 'TWO OR MORE RACES, INCLUDING MINORITY',
       'TWO OR MORE RACES, NON-MINORITIES', 'VISA NON U.S.',
       'CONTINUING GENERATION STUDENT', 'FIRST GENERATION STUDENT',
       'FIRST GENERATION UNKNOWN', 'DEP_FAM_1', 'DEP_FAM_10', 'DEP_FAM_11',
       'DEP_FAM_12', 'DEP_FAM_13', 'DEP_FAM_14', 'DEP_FAM_15', 'DEP_FAM_2',
       'DEP_FAM_25', 'DEP_FAM_3', 'DEP_FAM_4', 'DEP_FAM_5', 'DEP_FAM_6',
       'DEP_FAM_7', 'DEP_FAM_8', 'DEP_FAM_9', 'DEP_FAM_NA', 'URM_MINORITY',
   

In [60]:
dem.drop(labels=['GENDER', 'ETHNICITY', 'FIRST_GENERATION',
       'DEP_FAMILY_SIZE', 'MINORITY', 'APPLICANT_FAMILY_SIZE',
       'APPLICANT_INCOME', 'PELL_ELIGIBILITY'], axis=1, inplace=True)

### Create Time to Graduation Response Variables

Suggestion to team.  This dem_dif index should be done in the notebook rather than in SQL ahead of time.

In [61]:
dem['DEM_N'] = dem['DEM_DIFF_INDX'].apply(lambda x: x if (x >= 0) == False 
                             else int(x/5+1) if (x%10 == 0 or x%10 == 7) 
                             else int((x + 2)/5) )

dem['YRS_TO_GRAD'] = dem['DEM_N'] * 0.5

In [62]:
dem[['DEM_DIFF_INDX','YRS_TO_GRAD']].head(12)

,DEM_DIFF_INDX,YRS_TO_GRAD
0,NaN,NaN
1,50.0,5.5
2,38.0,4.0
3,38.0,4.0
4,48.0,5.0
5,48.0,5.0
6,NaN,NaN
7,NaN,NaN
8,50.0,5.5
9,48.0,5.0


In [63]:
dem = pd.concat([dem,pd.get_dummies(dem['YRS_TO_GRAD'], drop_first=False, prefix='GRAD_IN')], axis=1)

In [64]:
dem.drop(labels=['DEM_DIFF_INDX','DEM_N','DAE_EMPLID','PELLTOT_EMPLID','ESA_EMPLID',
                 'YRS_TO_GRAD'], axis=1, inplace=True)

KeyError: "['PELLTOT_EMPLID'] not found in axis"

In [65]:
dem.columns

Index(['EMPLID', 'DEM_COHORT', 'DEM_DIFF_INDX', 'DAE_EMPLID',
       'PELL_TOT_EMPLID', 'ESA_EMPLID', 'ACT_COMP', 'ACT_READ', 'ACT_MATH',
       'ACT_ENG', 'ACT_SCI', 'SAT_READ', 'SAT_MATH', 'SAT_COMP', 'GPA_HS',
       'GENDR_M', 'AFRICAN AMERICAN', 'ASIAN AMERICAN', 'CAUCASIAN',
       'ETHNICITY UNKNOWN', 'LATINO/LATINA', 'NATIVE AMERICAN',
       'PACIFIC ISLANDER', 'TWO OR MORE RACES, INCLUDING MINORITY',
       'TWO OR MORE RACES, NON-MINORITIES', 'VISA NON U.S.',
       'CONTINUING GENERATION STUDENT', 'FIRST GENERATION STUDENT',
       'FIRST GENERATION UNKNOWN', 'DEP_FAM_1', 'DEP_FAM_10', 'DEP_FAM_11',
       'DEP_FAM_12', 'DEP_FAM_13', 'DEP_FAM_14', 'DEP_FAM_15', 'DEP_FAM_2',
       'DEP_FAM_25', 'DEP_FAM_3', 'DEP_FAM_4', 'DEP_FAM_5', 'DEP_FAM_6',
       'DEP_FAM_7', 'DEP_FAM_8', 'DEP_FAM_9', 'DEP_FAM_NA', 'URM_MINORITY',
       'URM_NON-MINORITY', 'URM_UNKNOWN', 'URM_VISA NON U.S.', 'APP_FAM_1',
       'APP_FAM_2', 'APP_FAM_3', 'APP_FAM_4', 'APP_FAM_5', 'APP_FAM_6',
       '

In [ ]:
dem.columns

### Join the Demographic Data with the CSULB Academic Performance Data

In [66]:
dem = dem.rename(columns={'DEM_COHORT':'COHORT'})

In [67]:
dem.columns

Index(['EMPLID', 'COHORT', 'DEM_DIFF_INDX', 'DAE_EMPLID', 'PELL_TOT_EMPLID',
       'ESA_EMPLID', 'ACT_COMP', 'ACT_READ', 'ACT_MATH', 'ACT_ENG', 'ACT_SCI',
       'SAT_READ', 'SAT_MATH', 'SAT_COMP', 'GPA_HS', 'GENDR_M',
       'AFRICAN AMERICAN', 'ASIAN AMERICAN', 'CAUCASIAN', 'ETHNICITY UNKNOWN',
       'LATINO/LATINA', 'NATIVE AMERICAN', 'PACIFIC ISLANDER',
       'TWO OR MORE RACES, INCLUDING MINORITY',
       'TWO OR MORE RACES, NON-MINORITIES', 'VISA NON U.S.',
       'CONTINUING GENERATION STUDENT', 'FIRST GENERATION STUDENT',
       'FIRST GENERATION UNKNOWN', 'DEP_FAM_1', 'DEP_FAM_10', 'DEP_FAM_11',
       'DEP_FAM_12', 'DEP_FAM_13', 'DEP_FAM_14', 'DEP_FAM_15', 'DEP_FAM_2',
       'DEP_FAM_25', 'DEP_FAM_3', 'DEP_FAM_4', 'DEP_FAM_5', 'DEP_FAM_6',
       'DEP_FAM_7', 'DEP_FAM_8', 'DEP_FAM_9', 'DEP_FAM_NA', 'URM_MINORITY',
       'URM_NON-MINORITY', 'URM_UNKNOWN', 'URM_VISA NON U.S.', 'APP_FAM_1',
       'APP_FAM_2', 'APP_FAM_3', 'APP_FAM_4', 'APP_FAM_5', 'APP_FAM_6',
       'APP_

##### This is directly analagous to a SQL merge

In [68]:
supreme = pd.merge(dem, grd, on=['EMPLID','COHORT'], how='left')

In [ ]:
#pandas_profiling.ProfileReport(supreme)

In [ ]:
#pandas_profiling.ProfileReport(supreme).get_rejected_variables()

In [70]:
supreme.drop(labels=[ 'DFW',
#                       'AP',
                      'BCMP_UNITS_FOR_CREDIT',
                      'BCMP_UNITS_TAKEN',
                      'CUM_BCMP_UNITS_FOR_CREDIT',
                      'CUM_BCMP_UNITS_IN_GPA',
                      'CUM_BCMP_UNITS_TAKEN',
                      'CUM_UNITS_FOR_CREDIT',
                      'INCM_NO RESPONSE',
                      'UNITS_FOR_CREDIT',
                      'URM_UNKNOWN',
                      'URM_VISA NON U.S.'], axis=1, inplace=True)

In [71]:
pd.options.display.max_seq_items = supreme.columns.shape[0]

supreme.columns

Index(['EMPLID', 'COHORT', 'DEM_DIFF_INDX', 'DAE_EMPLID', 'PELL_TOT_EMPLID',
       'ESA_EMPLID', 'ACT_COMP', 'ACT_READ', 'ACT_MATH', 'ACT_ENG', 'ACT_SCI',
       'SAT_READ', 'SAT_MATH', 'SAT_COMP', 'GPA_HS', 'GENDR_M',
       'AFRICAN AMERICAN', 'ASIAN AMERICAN', 'CAUCASIAN', 'ETHNICITY UNKNOWN',
       'LATINO/LATINA', 'NATIVE AMERICAN', 'PACIFIC ISLANDER',
       'TWO OR MORE RACES, INCLUDING MINORITY',
       'TWO OR MORE RACES, NON-MINORITIES', 'VISA NON U.S.',
       'CONTINUING GENERATION STUDENT', 'FIRST GENERATION STUDENT',
       'FIRST GENERATION UNKNOWN', 'DEP_FAM_1', 'DEP_FAM_10', 'DEP_FAM_11',
       'DEP_FAM_12', 'DEP_FAM_13', 'DEP_FAM_14', 'DEP_FAM_15', 'DEP_FAM_2',
       'DEP_FAM_25', 'DEP_FAM_3', 'DEP_FAM_4', 'DEP_FAM_5', 'DEP_FAM_6',
       'DEP_FAM_7', 'DEP_FAM_8', 'DEP_FAM_9', 'DEP_FAM_NA', 'URM_MINORITY',
       'URM_NON-MINORITY', 'APP_FAM_1', 'APP_FAM_2', 'APP_FAM_3', 'APP_FAM_4',
       'APP_FAM_5', 'APP_FAM_6', 'APP_FAM_7', 'APP_FAM_8', 'APP_FAM_9',
       'A

### ACT, SAT and HS GPA Scores Preprocessing - Simple Imputation

Students may have ACT or SAT or both scores. The idea is to create a feature that would capture test performance in a general sense. The approach use here is to create three features that capture performance in Math, Reading and Composite performance. To this end we scale and center both ACT and SAT test scores in math, reading and composite and, in the event a student has taken both,choose the maximum normalized score.

##### Need to impute missing values of ACT and SAT scores:

Before this preprocessing step is undertaken it is necessary to split the student data into trainning and development sets in order to avoid "leakeage" from trainning into development since the imputing calculations and methods use the entire dataset.

In [78]:
supreme['N'].head()

0    1.0
1    2.0
2    3.0
3    4.0
4    4.0
Name: N, dtype: float64

In [79]:
supreme_train = pd.merge(students_train, supreme, on='EMPLID', how='inner')
print(supreme_train.shape)

supreme_dev = pd.merge(students_dev, supreme, on='EMPLID', how='inner')
print(supreme_dev.shape)

(103729, 116)
(11633, 116)


In [82]:
supreme_train[['ACT_COMP', 'ACT_READ', 'ACT_MATH', 'ACT_ENG', 'ACT_SCI', 'SAT_READ',
       'SAT_MATH', 'SAT_COMP', 'GPA_HS']].head()

,ACT_COMP,ACT_READ,ACT_MATH,ACT_ENG,ACT_SCI,SAT_READ,SAT_MATH,SAT_COMP,GPA_HS
0,NaN,NaN,NaN,NaN,NaN,570.0,700.0,1270.0,336.0
1,NaN,NaN,NaN,NaN,NaN,570.0,700.0,1270.0,336.0
2,NaN,NaN,NaN,NaN,NaN,570.0,700.0,1270.0,336.0
3,NaN,NaN,NaN,NaN,NaN,570.0,700.0,1270.0,336.0
4,NaN,NaN,NaN,NaN,NaN,570.0,700.0,1270.0,336.0


In [83]:
supreme_train['N'].head()

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: N, dtype: float64

##### Scale the scores and choose the max

In [87]:
supreme_train[['ACT_COMP', 'ACT_READ', 'ACT_MATH', 
         'ACT_ENG', 'ACT_SCI', 'SAT_READ',
         'SAT_MATH', 'SAT_COMP']].mean()

ACT_COMP      21.390909
ACT_READ      21.526857
ACT_MATH      22.255238
ACT_ENG       20.776264
ACT_SCI       20.969235
SAT_READ     500.680120
SAT_MATH     524.491361
SAT_COMP    1025.171481
dtype: float64
ACT_COMP      4.217409
ACT_READ      5.326887
ACT_MATH      4.601304
ACT_ENG       5.139600
ACT_SCI       4.229194
SAT_READ     83.188421
SAT_MATH     90.264058
SAT_COMP    154.787079
dtype: float64


In [85]:
supreme_train[['ACT_COMP', 'ACT_READ', 'ACT_MATH', 
         'ACT_ENG', 'ACT_SCI', 'SAT_READ',
         'SAT_MATH', 'SAT_COMP']].std()

ACT_COMP      4.217409
ACT_READ      5.326887
ACT_MATH      4.601304
ACT_ENG       5.139600
ACT_SCI       4.229194
SAT_READ     83.188421
SAT_MATH     90.264058
SAT_COMP    154.787079
dtype: float64

In [90]:
supreme_train[['ACT_COMP', 'ACT_READ', 'ACT_MATH', 
         'ACT_ENG', 'ACT_SCI', 'SAT_READ',
         'SAT_MATH', 'SAT_COMP']] = preprocessing.scale(supreme_train[['ACT_COMP', 'ACT_READ', 'ACT_MATH', 
                                                                         'ACT_ENG', 'ACT_SCI', 'SAT_READ', 
                                                                         'SAT_MATH', 'SAT_COMP']])

In [91]:
supreme_dev[['ACT_COMP', 'ACT_READ', 'ACT_MATH', 
         'ACT_ENG', 'ACT_SCI', 'SAT_READ',
         'SAT_MATH', 'SAT_COMP']] = preprocessing.scale(supreme_dev[['ACT_COMP', 'ACT_READ', 'ACT_MATH', 
                                                                         'ACT_ENG', 'ACT_SCI', 'SAT_READ', 
                                                                         'SAT_MATH', 'SAT_COMP']])

In [92]:
supreme_train[['ACT_COMP', 'ACT_READ', 'ACT_MATH', 'ACT_ENG', 'ACT_SCI', 'SAT_READ',
       'SAT_MATH', 'SAT_COMP']].mean()

ACT_COMP    1.718384e-16
ACT_READ    6.411353e-16
ACT_MATH    3.174167e-16
ACT_ENG    -2.290809e-16
ACT_SCI     1.808943e-16
SAT_READ    4.357727e-17
SAT_MATH    4.380824e-16
SAT_COMP    5.399021e-16
dtype: float64

In [93]:
supreme_train[['ACT_COMP', 'ACT_READ', 'ACT_MATH', 'ACT_ENG', 'ACT_SCI', 'SAT_READ',
       'SAT_MATH', 'SAT_COMP']].std()


ACT_COMP    1.000014
ACT_READ    1.000014
ACT_MATH    1.000014
ACT_ENG     1.000014
ACT_SCI     1.000014
SAT_READ    1.000005
SAT_MATH    1.000005
SAT_COMP    1.000005
dtype: float64

In [95]:
supreme_train['T_COMP'] = supreme_train[['ACT_COMP','SAT_COMP']].apply(lambda x: x.max(), axis=1)
supreme_train['T_READ'] = supreme_train[['ACT_READ','SAT_READ']].apply(lambda x: x.max(), axis=1)
supreme_train['T_MATH'] = supreme_train[['ACT_MATH','SAT_MATH']].apply(lambda x: x.max(), axis=1)

In [96]:
supreme_dev['T_COMP'] = supreme_dev[['ACT_COMP','SAT_COMP']].apply(lambda x: x.max(), axis=1)
supreme_dev['T_READ'] = supreme_dev[['ACT_READ','SAT_READ']].apply(lambda x: x.max(), axis=1)
supreme_dev['T_MATH'] = supreme_dev[['ACT_MATH','SAT_MATH']].apply(lambda x: x.max(), axis=1)

In [97]:
supreme_train[['T_COMP','T_READ','T_MATH']].isnull().sum()

T_COMP    1127
T_READ    1127
T_MATH    1127
dtype: int64

##### For now impute values by using the mean

Note to team: I think this should come before we scale

In [100]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [101]:
test_scores = supreme_train[['T_COMP','T_READ','T_MATH','GPA_HS']]

imp_mean.fit(test_scores)  

supreme_train[['T_COMP','T_READ','T_MATH','GPA_HS']] = imp_mean.transform(test_scores)

In [102]:
test_scores = supreme_dev[['T_COMP','T_READ','T_MATH','GPA_HS']]

imp_mean.fit(test_scores)  

supreme_dev[['T_COMP','T_READ','T_MATH','GPA_HS']] = imp_mean.transform(test_scores)

##### Drop unecessary features

In [103]:
supreme_train['N'].head()

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: N, dtype: float64

In [104]:
supreme_train.columns[30:50]

Index(['DEP_FAM_10', 'DEP_FAM_11', 'DEP_FAM_12', 'DEP_FAM_13', 'DEP_FAM_14',
       'DEP_FAM_15', 'DEP_FAM_2', 'DEP_FAM_25', 'DEP_FAM_3', 'DEP_FAM_4',
       'DEP_FAM_5', 'DEP_FAM_6', 'DEP_FAM_7', 'DEP_FAM_8', 'DEP_FAM_9',
       'DEP_FAM_NA', 'URM_MINORITY', 'URM_NON-MINORITY', 'APP_FAM_1',
       'APP_FAM_2'],
      dtype='object')

In [105]:
supreme_train.drop(['ACT_COMP', 'ACT_READ', 'ACT_MATH', 'ACT_ENG', 'ACT_SCI', 'SAT_READ',
       'SAT_MATH', 'SAT_COMP'], axis=1, inplace=True)

supreme_dev.drop(['ACT_COMP', 'ACT_READ', 'ACT_MATH', 'ACT_ENG', 'ACT_SCI', 'SAT_READ',
       'SAT_MATH', 'SAT_COMP'], axis=1, inplace=True)

### Normalization using Quantile Transformation (not used)

In [ ]:
# def norm_test(df):

#             qt_normal_scaler = preprocessing.QuantileTransformer(output_distribution = 'normal', random_state=0)

#             df_num = df[['CUM_GPA', 'BCMP_TERM_GPA', 'BCMP_CUM_GPA','DFW_RATE', 'T_COMP', 'T_READ', 'T_MATH', 'GPA_HS']]

#             quantiletransformation_normal = qt_normal_scaler.fit_transform(df_num)
#             quantiletransformation_normal_df = pd.DataFrame(quantiletransformation_normal ,columns=['CUM_GPA', 'BCMP_TERM_GPA', 'BCMP_CUM_GPA','DFW_RATE', 'T_COMP', 'T_READ', 'T_MATH', 'GPA_HS'])
       
            
            
#             df.loc[:,['CUM_GPA', 'BCMP_TERM_GPA', 'BCMP_CUM_GPA','DFW_RATE', 'T_COMP', 'T_READ', 'T_MATH', 'GPA_HS']] = quantiletransformation_normal_df.loc[:,['CUM_GPA', 'BCMP_TERM_GPA', 'BCMP_CUM_GPA','DFW_RATE', 'T_COMP', 'T_READ', 'T_MATH', 'GPA_HS']]
        
        
            
#             return df

In [ ]:
supreme_train.columns

### Normalization of Performance features

In [106]:
supreme_train[['CUM_BCMP', 'CUM_SUMMER', 'CUM_WINTER', 
               'TERM_GPA', 'CUM_GPA','BCMP_TERM_GPA', 
               'BCMP_CUM_GPA']] = preprocessing.scale(supreme_train[['CUM_BCMP', 'CUM_SUMMER', 'CUM_WINTER', 
                                                                     'TERM_GPA', 'CUM_GPA','BCMP_TERM_GPA', 'BCMP_CUM_GPA']])

In [107]:
supreme_dev[['CUM_BCMP', 'CUM_SUMMER', 'CUM_WINTER', 
               'TERM_GPA', 'CUM_GPA','BCMP_TERM_GPA', 
               'BCMP_CUM_GPA']] = preprocessing.scale(supreme_dev[['CUM_BCMP', 'CUM_SUMMER', 'CUM_WINTER', 
                                                                     'TERM_GPA', 'CUM_GPA','BCMP_TERM_GPA', 'BCMP_CUM_GPA']])

In [108]:
print(
    
    supreme_train[['CUM_BCMP', 'CUM_SUMMER', 'CUM_WINTER', 
               'TERM_GPA', 'CUM_GPA','BCMP_TERM_GPA', 
               'BCMP_CUM_GPA']].mean()
)

print(
    
    supreme_train[['CUM_BCMP', 'CUM_SUMMER', 'CUM_WINTER', 
               'TERM_GPA', 'CUM_GPA','BCMP_TERM_GPA', 
               'BCMP_CUM_GPA']].std()
)

CUM_BCMP        -1.717918e-15
CUM_SUMMER      -2.555755e-15
CUM_WINTER       3.299559e-15
TERM_GPA        -1.632295e-15
CUM_GPA         -7.642108e-16
BCMP_TERM_GPA   -8.256939e-15
BCMP_CUM_GPA     3.720508e-16
dtype: float64
CUM_BCMP         1.000005
CUM_SUMMER       1.000005
CUM_WINTER       1.000005
TERM_GPA         1.000005
CUM_GPA          1.000005
BCMP_TERM_GPA    1.000005
BCMP_CUM_GPA     1.000005
dtype: float64


##### The AP feature is null - impute with a constant for now

Note to team:  I reccomend we drop AP from this presentation data entirely.

In [109]:
supreme_train['AP'] = 0

supreme_dev['AP'] = 0

In [110]:
supreme_train.isnull().sum().head(55)

EMPLID                                       0
COHORT                                       0
DEM_DIFF_INDX                            17566
DAE_EMPLID                                   0
PELL_TOT_EMPLID                              0
ESA_EMPLID                                   0
GPA_HS                                       0
GENDR_M                                      0
AFRICAN AMERICAN                             0
ASIAN AMERICAN                               0
CAUCASIAN                                    0
ETHNICITY UNKNOWN                            0
LATINO/LATINA                                0
NATIVE AMERICAN                              0
PACIFIC ISLANDER                             0
TWO OR MORE RACES, INCLUDING MINORITY        0
TWO OR MORE RACES, NON-MINORITIES            0
VISA NON U.S.                                0
CONTINUING GENERATION STUDENT                0
FIRST GENERATION STUDENT                     0
FIRST GENERATION UNKNOWN                     0
DEP_FAM_1    

### Create the Major Plan Change Index

##### Academic plans are 10 characters in length

In [ ]:
supreme_train['EOT_ACAD_PLAN_CD']

In [ ]:
supreme_train['EOT_ACAD_PLAN_CD'] = supreme_train['EOT_ACAD_PLAN_CD'].apply(lambda x: str(x))
supreme_dev['EOT_ACAD_PLAN_CD'] = supreme_dev['EOT_ACAD_PLAN_CD'].apply(lambda x: str(x))

In [ ]:
supreme_train['EOT_ACAD_PLAN_CD'].apply(lambda x: len(x)).describe()

##### The first 4 characters representthe department/subject. The 5th and 6th character encode the plan type and allows to separate pre-majors from majors. A change from pre-major to major in the same department/subject is not considered a major change.

In [ ]:
print (
    supreme_train['EOT_ACAD_PLAN_CD'].apply(lambda x: x[4:6]).head()
)

print (
    supreme_train['EOT_ACAD_PLAN_CD'].apply(lambda x: x[:4]).head()
)

##### Create codes

In [ ]:
supreme_train['SUBJECT'] = supreme_train['EOT_ACAD_PLAN_CD'].apply(lambda x: x[:4])
supreme_train['PLAN_TYPE'] = supreme_train['EOT_ACAD_PLAN_CD'].apply(lambda x: x[4:6])

In [ ]:
supreme_dev['SUBJECT'] = supreme_dev['EOT_ACAD_PLAN_CD'].apply(lambda x: x[:4])
supreme_dev['PLAN_TYPE'] = supreme_dev['EOT_ACAD_PLAN_CD'].apply(lambda x: x[4:6])

In [ ]:
supreme_train[['EMPLID','SUBJECT','PLAN_TYPE']].head()

##### Create Major Change indicator

In [ ]:
supreme_train['MAJOR_CHANGE_INDICATOR'] = ( supreme_train[['EMPLID','SUBJECT','PLAN_TYPE']] == supreme_train[['EMPLID','SUBJECT','PLAN_TYPE']].shift() ).apply(
    
    lambda x: 0 if x[0] == False 
           
           or ( x[0] == True and x[1] == True and x[2] == True ) 
           
           or ( x[0] == True and x[1] == True and x[2] == False )
           
           else 1 if ( x[0] == True and x[1] == False and x[2] == True )
           
           or ( x[0] == True and x[1] == False and x[2] == False )
           
           else None,
           
           axis=1)

In [ ]:
supreme_dev['MAJOR_CHANGE_INDICATOR'] = ( supreme_dev[['EMPLID','SUBJECT','PLAN_TYPE']] == supreme_dev[['EMPLID','SUBJECT','PLAN_TYPE']].shift() ).apply(
    
    lambda x: 0 if x[0] == False 
           
           or ( x[0] == True and x[1] == True and x[2] == True ) 
           
           or ( x[0] == True and x[1] == True and x[2] == False )
           
           else 1 if ( x[0] == True and x[1] == False and x[2] == True )
           
           or ( x[0] == True and x[1] == False and x[2] == False )
           
           else None,
           
           axis=1)


##### Create a cumulative Major Change Counter

In [ ]:
grouped_cumsum = supreme_train[['EMPLID',
                             'TERM_CODE',
                             'EOT_ACAD_PLAN_CD',
                             'COHORT',
                             'MAJOR_CHANGE_INDICATOR'
                             ]].groupby(['EMPLID',
                                                 'TERM_CODE',
                                                 'EOT_ACAD_PLAN_CD',
                                                 'COHORT',
                                                 ]).sum().groupby(level=[0]).cumsum().reset_index()

grouped_cumsum = grouped_cumsum.add_prefix('CUM_')

supreme_train = pd.concat([supreme_train,grouped_cumsum['CUM_MAJOR_CHANGE_INDICATOR']],axis=1)

supreme_train.rename(columns={'CUM_MAJOR_CHANGE_INDICATOR':'MAJOR_CHANGE_CNT','N':'SEMESTER_INDEX'}, inplace=True)

In [ ]:
grouped_cumsum = supreme_dev[['EMPLID',
                             'TERM_CODE',
                             'EOT_ACAD_PLAN_CD',
                             'COHORT',
                             'MAJOR_CHANGE_INDICATOR'
                             ]].groupby(['EMPLID',
                                                 'TERM_CODE',
                                                 'EOT_ACAD_PLAN_CD',
                                                 'COHORT',
                                                 ]).sum().groupby(level=[0]).cumsum().reset_index()

grouped_cumsum = grouped_cumsum.add_prefix('CUM_')

supreme_dev = pd.concat([supreme_dev,grouped_cumsum['CUM_MAJOR_CHANGE_INDICATOR']],axis=1)

supreme_dev.rename(columns={'CUM_MAJOR_CHANGE_INDICATOR':'MAJOR_CHANGE_CNT','N':'SEMESTER_INDEX'}, inplace=True)

##### Rearrange features

In [ ]:
supreme_train.columns

In [ ]:
id_var = ['COHORT', 'EMPLID']

perf_var = ['TERM_CODE', 'SEMESTER_INDEX', 'EOT_ACAD_PLAN_CD', 'MAJOR_CHANGE_INDICATOR','MAJOR_CHANGE_CNT',
            'UNITS_TAKEN','BCMP', 'A', 'AU', 'B' ,'C','CR', 'D', 'F', 'I', 'NC', 'RP', 'W', 'WE', 'WU', 'SUMMER', 
            'CUM_BCMP', 'CUM_SUMMER', 'CUM_WINTER', 'TERM_GPA','CUM_GPA', 'BCMP_TERM_GPA', 'BCMP_CUM_GPA',
            'COMPLETION_RATE', 'LOAD_INDEX_EXCLUDE', 'LOAD_INDEX_ONLY','DFW_RATE','T_COMP', 'T_READ','T_MATH']

dem_var = ['GPA_HS', 'GENDR_M', 'AFRICAN AMERICAN','ASIAN AMERICAN', 'CAUCASIAN', 'LATINO/LATINA', 
           'NATIVE AMERICAN','PACIFIC ISLANDER', 'TWO OR MORE RACES, INCLUDING MINORITY',
           'TWO OR MORE RACES, NON-MINORITIES', 'ETHNICITY UNKNOWN', 'VISA NON U.S.','CONTINUING GENERATION STUDENT', 
           'FIRST GENERATION STUDENT', 'FIRST GENERATION UNKNOWN','DEP_FAM_1', 'DEP_FAM_10', 'DEP_FAM_12', 'DEP_FAM_15', 'DEP_FAM_2',
           'DEP_FAM_25', 'DEP_FAM_3', 'DEP_FAM_4', 'DEP_FAM_5', 'DEP_FAM_6','DEP_FAM_7', 'DEP_FAM_8', 'DEP_FAM_9', 
           'DEP_FAM_NA', 'URM_MINORITY','URM_NON-MINORITY', 'APP_FAM_1',
           'APP_FAM_2', 'APP_FAM_3', 'APP_FAM_6', 'APP_FAM_NA','INCM_$12,000 TO $23,999', 'INCM_$6,000 TO $11,999',
           'INCM_$60,000 OR MORE', 'INCM_LESS THEN $6000', 'PELL_NON TRADITIONAL', 
           'PELL_TRADITIONAL']

resp_var = ['GRAD_IN_2.5','GRAD_IN_3.0', 'GRAD_IN_3.5', 'GRAD_IN_4.0', 'GRAD_IN_4.5','GRAD_IN_5.0', 'GRAD_IN_5.5', 
          'GRAD_IN_6.0']

In [ ]:
supreme_train = supreme_train[id_var + perf_var + dem_var + resp_var]
supreme_dev = supreme_dev[id_var + perf_var + dem_var + resp_var]

In [ ]:
for feature in supreme_train.columns: print(feature)

In [ ]:
supreme_train[['SUMMER','CUM_SUMMER','CUM_WINTER']].head(9)

In [ ]:
supreme_train[supreme_train['EMPLID'] == '011193375']

### Descripte Stats

In [ ]:
supreme_train['GPA_HS'].describe()

In [ ]:
pd.to_numeric(supreme['TERM_CODE']).describe()

In [ ]:
supreme_dev['BCMP_TERM_GPA'].std()

### Define Sequential Data Chunks

In [ ]:
# Clean up semester_index as to eliminate floats
print(supreme_train['SEMESTER_INDEX'].unique())
supreme_train.dropna(inplace=True)
supreme_train['SEMESTER_INDEX'] = supreme_train['SEMESTER_INDEX'].astype('int32')

In [ ]:
#Should Dev also be cleaned up to eliminate floats?
print(supreme_dev['SEMESTER_INDEX'].unique())
supreme_dev['SEMESTER_INDEX'] = supreme_dev['SEMESTER_INDEX'].astype('int32')

In [ ]:
supreme_dev.head()

In [ ]:
supreme_train.head()

In [ ]:
#Change to numeric type
supreme_train['TERM_CODE'] = pd.to_numeric(supreme_train['TERM_CODE'])
supreme_dev['TERM_CODE'] = pd.to_numeric(supreme_dev['TERM_CODE'])

In [ ]:
supreme_train[supreme_train['EMPLID'] == '010841881'][['EMPLID', 'SEMESTER_INDEX', 'TERM_GPA']].head()

In [ ]:
seq = {'TRAIN0':supreme_train[id_var + dem_var + resp_var]}
seq['DEV0'] = supreme_dev[id_var + dem_var + resp_var]

seq['TRAIN0'].drop_duplicates(inplace=True)

seq['DEV0'].drop_duplicates(inplace=True)

In [ ]:
for s in supreme_train['SEMESTER_INDEX'].unique():
    seq['TRAIN{}'.format(s)] = supreme_train[supreme_train['SEMESTER_INDEX'] <= s]
    seq['DEV{}'.format(s)] = supreme_dev[supreme_dev['SEMESTER_INDEX'] <= s]

In [ ]:
seq.keys()

In [ ]:
print(seq['TRAIN0'].shape)
print(seq['TRAIN1'].shape)
print(seq['TRAIN2'].shape)
print(seq['TRAIN3'].shape)

In [ ]:
print(seq['TRAIN0']['GRAD_IN_4.0'].mean())

In [ ]:
# supreme_dev.to_csv('supreme_dev.csv')

# supreme_train.to_csv('supreme_train.csv')

# Modelling

In [ ]:
def model_tuner(X_train,y_train,X_dev,y_dev,model,grid = None):
    
    if grid == None:
        clf = model
    else:
        clf = GridSearchCV(model, grid, cv=10, n_jobs = -1)
    
    clf_fit = clf.fit(X_train,y_train)
    
    if grid != None: 
        best_par = clf.best_params_
    
    y_dev_pred = clf.predict(X_dev)
    y_train_pred = clf.predict(X_train)
    p_pred = clf.predict(X_dev)
    cm = confusion_matrix(y_dev,y_dev_pred)
    dev_accuracy = accuracy_score(y_dev,y_dev_pred)
    train_accuracy = accuracy_score(y_train,y_train_pred)
    report = classification_report(y_dev,y_dev_pred)
    
    if grid != None: 
        print ('\nthe optimal parameters are: {}'.format(best_par))
    
    print ('\naccuracy on the dev set is: {}'.format(dev_accuracy))
    print ('\naccuracy on the train set is: {}'.format(train_accuracy))
    print ('\nconfusion matrix:\n\n{}'.format(cm))
    print ('\nclassification report:\n\n{}'.format(report))
    
    if grid != None:
        results_dict = {'best model':clf_fit,'best parameters':best_par, 'predicted dev values':y_dev_pred, 
                        'predicted training values':y_train_pred,'predicted probabilities':p_pred,
                        'confusion matrix':cm,'dev accuracy':dev_accuracy,
                        'training accuracy':train_accuracy,'classification report':report}
    else:
        results_dict = {'best model':clf_fit, 'predicted dev values':y_dev_pred, 'predicted training values':y_train_pred, 
                        'predicted probabilities':p_pred,'confusion matrix':cm,'dev accuracy':dev_accuracy,
                        'training accuracy':train_accuracy,'classification report':report}
    return results_dict

In [ ]:
var = [
    
       'MAJOR_CHANGE_CNT', 'SUMMER','TERM_GPA','BCMP_TERM_GPA','COMPLETION_RATE', 'LOAD_INDEX_EXCLUDE', 
       'LOAD_INDEX_ONLY', 'DFW_RATE', 'GPA_HS','GENDR_M', 'AFRICAN AMERICAN',
       'ASIAN AMERICAN', 'CAUCASIAN', 'LATINO/LATINA', 'NATIVE AMERICAN', 'PACIFIC ISLANDER', 
       'TWO OR MORE RACES, INCLUDING MINORITY',
       'TWO OR MORE RACES, NON-MINORITIES', 'ETHNICITY UNKNOWN', 'VISA NON U.S.',
       'CONTINUING GENERATION STUDENT', 'FIRST GENERATION STUDENT', 'FIRST GENERATION UNKNOWN',
       'DEP_FAM_1', 'DEP_FAM_10', 'DEP_FAM_12', 'DEP_FAM_15', 'DEP_FAM_2',
       'DEP_FAM_25', 'DEP_FAM_3', 'DEP_FAM_4', 'DEP_FAM_5', 'DEP_FAM_6',
       'DEP_FAM_7', 'DEP_FAM_8', 'DEP_FAM_9', 'DEP_FAM_NA', 'URM_MINORITY',
       'URM_NON-MINORITY', 'APP_FAM_1',
       'APP_FAM_2', 'APP_FAM_3', 'APP_FAM_6', 'APP_FAM_NA',
       'INCM_$12,000 TO $23,999', 'INCM_$6,000 TO $11,999',
       'INCM_$60,000 OR MORE', 'INCM_LESS THEN $6000', 
       'PELL_NON TRADITIONAL', 'PELL_TRADITIONAL', 'T_COMP', 'T_READ',
       'T_MATH'
    
]


var2 = [
    
       'MAJOR_CHANGE_CNT', 'A', 'AU', 'B',
       'C', 'CR', 'D', 'F', 'I', 'NC', 'RP', 'W', 'WE', 'WU', 'SUMMER',
       'WINTER', 'CUM_SUMMER', 'CUM_WINTER', 'CUM_GPA', 'BCMP_CUM_GPA', 'COMPLETION_RATE', 
       'LOAD_INDEX_EXCLUDE', 'LOAD_INDEX_ONLY', 'DFW_RATE', 'GPA_HS'
    
]

var3 = [
    
       'MAJOR_CHANGE_CNT', 'SUMMER',
       'WINTER', 'CUM_SUMMER', 'CUM_WINTER', 'CUM_GPA', 'BCMP_CUM_GPA', 
       'LOAD_INDEX_EXCLUDE', 'LOAD_INDEX_ONLY', 'DFW_RATE', 'GPA_HS'
    
]

In [ ]:
t=0
time_to_grad=4

train_t = 'TRAIN{}'.format(str(t))
dev_t = 'DEV{}'.format(str(t))
response = 'GRAD_IN_{}.0'.format(time_to_grad)

y_train = seq[train_t][response]
y_train.head()

In [ ]:
y_dev = seq[dev_t][['COHORT','EMPLID',response]]
y_dev.head()

In [ ]:
X_train = seq[train_t][dem_var]
X_train.head()

In [ ]:
train_col = X_train.columns

In [ ]:
X_dev = seq[dev_t][dem_var]
X_train, y_train = SMOTE().fit_sample(X_train, y_train)
#X_dev, y_dev = SMOTE().fit_sample(X_dev, y_dev)

X_train

In [ ]:
pd.DataFrame(data=X_train,columns=train_col).head()

In [ ]:
def seq_data(t, time_to_grad, features):
    train_t = 'TRAIN{}'.format(str(t))
    dev_t = 'DEV{}'.format(str(t))
    response = 'GRAD_IN_{}.0'.format(time_to_grad)

    y_train = seq[train_t][response]
    y_dev = seq[dev_t][response]
    X_train = seq[train_t][features]
    X_dev = seq[dev_t][features]

    X_train, y_train = SMOTE().fit_sample(X_train, y_train)
    X_train = pd.DataFrame(data=X_train,columns=features)
    #X_dev, y_dev = SMOTE().fit_sample(X_dev, y_dev)
    
    return (X_train,y_train,X_dev,y_dev)

In [ ]:
def accuracy_matrix_fn(df, data):
    aMat = df
    aMat = aMat.append(pd.DataFrame(data), sort=False)
    aMat.reset_index(inplace=True,drop=True)
    return aMat

In [ ]:
def run_model(model,grid = None,label=None):
    n_features = len(X_train.columns)
    n_obs = len(y_train)
    start = time.time()
    results_log = model_tuner(X_train,y_train,X_dev,y_dev,model,grid)
    end = time.time()
    runtime = end - start
    print ('the runtime is {} minutes'.format(runtime/60))
    
    return {'model':label, 'Number of Features': n_features, 'Number of Obs': n_obs, 'dev accuracy':[results_log['dev accuracy']], 'training accuracy':[results_log['training accuracy']]}

In [ ]:
random_state = 42

### Lasso Variable Selection

In [ ]:

# def lasso_reduction(X_train, X_dev):
#     clf = LassoCV(cv=10, random_state=random_state)
#     clf.fit(X_train,y_train)
#     alpha = clf.alpha_
#     coef = clf.coef_
#     sfm = SelectFromModel(clf)
#     sfm.fit(X_train,y_train)
#     selection = sfm.get_support()
#     X_train = pd.DataFrame(sfm.transform(X_train), columns=X_dev.columns[selection])
#     X_dev = pd.DataFrame(X_dev.iloc[:,selection])
#     return X_train, X_dev, alpha, coef, selection

In [ ]:
# X_train,y_train,X_dev,y_dev = seq_data(t=1, time_to_grad=4, features=var)
# X_train, X_dev, alpha, coef, selection = lasso_reduction(X_train, X_dev)
# X_train.head()

##### Logistic at $t_0$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=0, time_to_grad=4, features=dem_var)

In [ ]:
X_train.head()

In [ ]:
log_mod = LogisticRegression(solver='lbfgs', max_iter=10000, random_state = random_state)

In [ ]:
t0_log_accuracy = run_model(model=log_mod,label='logistic @ t0')

In [ ]:
accuracy_matrix = pd.DataFrame()
accuracy_matrix = accuracy_matrix.append(pd.DataFrame(t0_log_accuracy), sort=False)

In [ ]:
accuracy_matrix.reset_index(inplace=True,drop=True)
accuracy_matrix

##### Logistic at $t_1$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=1, time_to_grad=4, features=var)

t1_log_accuracy = run_model(model=log_mod,label='logistic @ t1')

In [ ]:
accuracy_matrix = accuracy_matrix.append(pd.DataFrame(t1_log_accuracy), sort=False)
accuracy_matrix.reset_index(inplace=True,drop=True)
accuracy_matrix

##### Logistic at $t_2$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=2, time_to_grad=4, features=var)
t2_log_accuracy = run_model(model=log_mod,label='logistic @ t2')

In [ ]:
accuracy_matrix = accuracy_matrix.append(pd.DataFrame(t2_log_accuracy), sort=False)
accuracy_matrix.reset_index(inplace=True,drop=True)
accuracy_matrix

##### Logistic at $t_3$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=3, time_to_grad=4, features=var)
t3_log_accuracy = run_model(model=log_mod,label='logistic @ t3')

In [ ]:
accuracy_matrix = accuracy_matrix.append(pd.DataFrame(t3_log_accuracy), sort=False)
accuracy_matrix.reset_index(inplace=True,drop=True)
accuracy_matrix

##### Logistic at $t_4$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=4, time_to_grad=4, features=var)
t4_log_accuracy = run_model(model=log_mod,label='logistic @ t4')

In [ ]:
accuracy_matrix = accuracy_matrix.append(pd.DataFrame(t4_log_accuracy), sort=False)
accuracy_matrix.reset_index(inplace=True,drop=True)
accuracy_matrix

In [ ]:
ax = plt.gca()

accuracy_matrix.plot(kind='line',x='model',y='dev accuracy',ax=ax)
accuracy_matrix.plot(kind='line',x='model',y='training accuracy', color='red', ax=ax)

plt.show()

### Random Forest 

##### Random Forest @ $t_0$

In [ ]:
# X_train,y_train,X_dev,y_dev = seq_data(t=0, time_to_grad=4, features=dem_var)
# X_train, X_dev, alpha, coef, selection = lasso_reduction(X_train, X_dev)
# X_train.head()

In [ ]:
len(y_train)

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=0, time_to_grad=4, features=dem_var)


parameters_rf = {'n_estimators':[100,100],'max_features':[1,int(np.sqrt(len(dem_var)))],'max_depth':[10,10]}
rf = RandomForestClassifier(n_jobs=-1, random_state = random_state)

t0_rf_accuracy = run_model(model=rf,grid=parameters_rf,label='random forest @ t0')

accuracy_matrix = pd.DataFrame()
accuracy_matrix = accuracy_matrix.append(pd.DataFrame(t0_rf_accuracy), sort=False)
accuracy_matrix.reset_index(inplace=True,drop=True)
accuracy_matrix

##### Random Forest @ $t_1$

In [ ]:
# rf_var  = ['COHORT', 'EMPLID', 'TERM_CODE', 'SEMESTER_INDEX','MAJOR_CHANGE_INDICATOR', 'MAJOR_CHANGE_CNT', 'UNITS_TAKEN', 'BCMP',
#        'A', 'AU', 'B', 'C', 'CR', 'D', 'F', 'I', 'NC', 'RP', 'W', 'WE', 'WU',
#        'SUMMER', 'CUM_BCMP', 'CUM_SUMMER', 'CUM_WINTER', 'TERM_GPA', 'CUM_GPA',
#        'BCMP_TERM_GPA', 'BCMP_CUM_GPA', 'COMPLETION_RATE',
#        'LOAD_INDEX_EXCLUDE', 'LOAD_INDEX_ONLY', 'DFW_RATE', 'T_COMP', 'T_READ',
#        'T_MATH', 'GPA_HS', 'GENDR_M', 'AFRICAN AMERICAN', 'ASIAN AMERICAN',
#        'CAUCASIAN', 'LATINO/LATINA', 'NATIVE AMERICAN', 'PACIFIC ISLANDER',
#        'TWO OR MORE RACES, INCLUDING MINORITY',
#        'TWO OR MORE RACES, NON-MINORITIES', 'ETHNICITY UNKNOWN',
#        'VISA NON U.S.', 'CONTINUING GENERATION STUDENT',
#        'FIRST GENERATION STUDENT', 'FIRST GENERATION UNKNOWN', 'DEP_FAM_1',
#        'DEP_FAM_10', 'DEP_FAM_12', 'DEP_FAM_15', 'DEP_FAM_2', 'DEP_FAM_25',
#        'DEP_FAM_3', 'DEP_FAM_4', 'DEP_FAM_5', 'DEP_FAM_6', 'DEP_FAM_7',
#        'DEP_FAM_8', 'DEP_FAM_9', 'DEP_FAM_NA', 'URM_MINORITY',
#        'URM_NON-MINORITY', 'APP_FAM_1', 'APP_FAM_2', 'APP_FAM_3', 'APP_FAM_6',
#        'APP_FAM_NA', 'INCM_$12,000 TO $23,999', 'INCM_$6,000 TO $11,999',
#        'INCM_$60,000 OR MORE', 'INCM_LESS THEN $6000', 'PELL_NON TRADITIONAL',
#        'PELL_TRADITIONAL']

In [ ]:
# X_train,y_train,X_dev,y_dev = seq_data(t=1, time_to_grad=4, features=var)
# X_train, X_dev, alpha, coef, selection = lasso_reduction(X_train, X_dev)
# X_train.head()

In [ ]:
X_train.columns

In [ ]:
X_train.shape

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=1, time_to_grad=4, features=var)

parameters_rf = {'n_estimators':[100,100],'max_features':[1,int(np.sqrt(len(var)))],'max_depth':[1,32]}

t1_rf_accuracy = run_model(model=rf,grid=parameters_rf, label='random forest @ t1')

accuracy_matrix = accuracy_matrix.append(pd.DataFrame(t1_rf_accuracy), sort=False)
accuracy_matrix.reset_index(inplace=True,drop=True)
accuracy_matrix

##### Random Forest @ $t_2$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=2, time_to_grad=4, features=var)

t2_rf_accuracy = run_model(model=rf,grid=parameters_rf, label='random forest @ t2')

accuracy_matrix = accuracy_matrix.append(pd.DataFrame(t2_rf_accuracy), sort=False)
accuracy_matrix.reset_index(inplace=True,drop=True)
accuracy_matrix

##### Random Forest @ $t_3$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=3, time_to_grad=4, features=var)

t3_rf_accuracy = run_model(model=rf,grid=parameters_rf, label='random forest @ t3')

accuracy_matrix = accuracy_matrix.append(pd.DataFrame(t3_rf_accuracy), sort=False)
accuracy_matrix.reset_index(inplace=True,drop=True)
accuracy_matrix

In [ ]:
# ax = plt.gca()

# accuracy_matrix.plot(kind='line',x='model',y='dev accuracy',ax=ax)
# accuracy_matrix.plot(kind='line',x='model',y='training accuracy', color='red', ax=ax)

# plt.show()

### Feature Importance from Random Forest

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=3, time_to_grad=4, features=rf_var)


### XGBoost

##### XGBoost @ $t_0$

In [ ]:
def run_model_xgb(label,num_round):
    
    d_train = xgb.DMatrix(X_train, label=y_train)
    d_dev = xgb.DMatrix(X_dev, label=y_dev)
    
    param = {'max_depth': 6, 'eta': 0.6, 'objective': 'binary:logistic'}
    param['nthread'] = 4
    param['eval_metric'] = 'auc'
    evallist = [(d_dev, 'eval'), (d_train, 'train')]

    bst = xgb.train(param, d_train, num_round, evallist, verbose_eval=True)
    
    y_dev_pred = (bst.predict(d_dev) > 0.5) * 1
    y_train_pred = (bst.predict(d_train) > 0.5) * 1
    p_pred = bst.predict(d_dev)
    cm = confusion_matrix(y_dev,y_dev_pred)
    dev_accuracy = accuracy_score(y_dev,y_dev_pred)
    train_accuracy = accuracy_score(y_train,y_train_pred)
    dev_f1 = f1_score(y_dev,y_dev_pred)
    train_f1 = f1_score(y_train,y_train_pred)
    dev_precision = precision_score(y_dev,y_dev_pred)
    train_precision = precision_score(y_train,y_train_pred)
    dev_recall = recall_score(y_dev,y_dev_pred)
    train_recall = recall_score(y_train,y_train_pred)
    dev_roc = roc_auc_score(y_dev,y_dev_pred)
    train_roc = roc_auc_score(y_train,y_train_pred)
    report = classification_report(y_dev,y_dev_pred)
    
    print ('\naccuracy on the dev set is: {}'.format(dev_accuracy))
    print ('\naccuracy on the train set is: {}'.format(train_accuracy))
    print ('\nconfusion matrix:\n\n {}'.format(cm))
    print ('\nclassification report:\n\n{}'.format(report))
    
    metrics = {'model':label,
               'dev accuracy':[dev_accuracy], 
               'training accuracy':[train_accuracy],
               'dev f1':[dev_f1],
               'training f1':[train_f1],
               'dev precision':[dev_precision],
               'training precision':[train_precision],
               'dev recall':[dev_recall],
               'training recall':[train_recall],
               'dev roc':[dev_roc],
               'training roc':[train_roc]
              } 
    
    predict = {'predicted dev values':[y_dev_pred],
               'predicted training values':[y_train_pred],
               'predicted probabilities':[p_pred],
               'confusion matrix':[cm],
               'classification report':[report]}
    
    return (metrics, predict)

In [ ]:
epochs = 14

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=0, time_to_grad=4, features=dem_var)
metrics,_ = run_model_xgb('xgb@t0',epochs)

In [ ]:
acc_mat = accuracy_matrix_fn(pd.DataFrame(), metrics)
acc_mat

##### XGBoost @ $t_1$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=1, time_to_grad=4, features=var)

metrics,_ = run_model_xgb('xgb@t1',epochs)

In [ ]:
1-y_dev.mean()

In [ ]:
acc_mat = accuracy_matrix_fn(acc_mat,metrics)
acc_mat

##### XGBoost @ $t_2$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=2, time_to_grad=4, features=var)
metrics,_ = run_model_xgb('xgb@t2',epochs)

acc_mat = accuracy_matrix_fn(acc_mat,metrics)
acc_mat

##### XGBoost @ $t_3$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=3, time_to_grad=4, features=var)
metrics,_ = run_model_xgb('xgb@t3',epochs)

acc_mat = accuracy_matrix_fn(acc_mat,metrics)
acc_mat

##### XGBoost @ $t_4$

In [ ]:
X_train,y_train,X_dev,y_dev = seq_data(t=4, time_to_grad=4, features=var)
metrics,_ = run_model_xgb('xgb@t4',epochs)

acc_mat = accuracy_matrix_fn(acc_mat,metrics)
acc_mat

In [ ]:
seq['TRAIN2'].keys()

In [ ]:
X_train